In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setMaster("local[2]").setAppName("DegreesOfSeparation")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/05 15:00:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/05 15:00:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# The characters we wish to find the degree of separation between:
startCharacterID = 5306 # SpiderMan
targetCharacterID = 14 # adam 3,031

In [6]:
# Our accumulator, used to signal when we find the target character during our BFS traversal.
hitCounter = sc.accumulator(0)

In [7]:
def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = "WHITE"
    distance = 9999

    if (heroID == startCharacterID):
        color = "GRAY"
        distance = 0

    return (heroID, (connections, distance, color))

In [19]:
def createStartingRdd():
    inputFile = sc.textFile("resources/Marvel-Graph")
    return inputFile.map(convertToBFS)

In [30]:
def bfsMap(node):
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]

    results = []
    # if this node needs to be expanded
    if (color == "GRAY"):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = "GRAY"
            if (targetCharacterID == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)
        
        # we've processed this node, so color it black
        color = "BLACK"

    # emitting the input node so we don't lost it.
    results.append((characterID, (connections, distance, color)))
    return results

In [31]:
def bfsReduce(data1, data2):
    edges1 = data1[0]
    edges2 = data2[0]

    distance1 = data1[1]
    distance2 = data2[1]

    color1 = data1[2]
    color2 = data2[2]

    distance = 9999
    color = color1
    edges = []

    # see if one is the original node with its connections, if so preserve them
    if (len(edges1) > 0):
        edges.extend(edges1)
    if (len(edges2) > 0):
        edges.extend(edges2)

    # preserve minimum distance
    if (distance1 < distance):
        distance = distance1
    
    if (distance2 < distance):
        distance = distance2

    # preserve darkest color
    if (color1 == "WHITE" and (color2 == "GRAY" or color2 == "BLACK")):
        color = color2

    if (color1 == "GRAY" and color2 == "BLACK"):
        color = color2

    if (color2 == "WHITE" and (color1 == "GRAY" or color1 == "BLACK")):
        color = color1

    if (color2 == "GRAY" and color1 == "BLACK"):
        color = color1

    return (edges, distance, color)

In [2]:
# main program here:
iterationRdd = createStartingRdd()

In [3]:
for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))

    # creating new vertices as needed to darken or reduce distances in the 
    # reduce stage. If we encounter the node we're looking for as a GRAY node,
    # increment our accumulator to signal that we're done.
    mapped = iterationRdd.flatMap(bfsMap)

    # note thta mappend.count() action here forces the RDD to be evaluated, and 
    # that's that only reason our accumulator is actually updated.
    print("Preprocessing " + str(mapped.count()) + " values.")

    if (hitCounter.value > 0):
        print("Hit the target character! From " + str(hitCounter.value)\
            + " different direction(s).")
        break

    # reducer combined data for each character ID, preserving the darkest color and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)

Running BFS iteration# 1
Preprocessing 8330 values.
Hit the target character! From 1 different direction(s).
